# Data Preprocessing

This notebook performs data preprocessing on LendingClub Loan Data obtained from [Kaggle](https://www.kaggle.com/wordsforthewise/lending-club).

In [1]:
import numpy as np
import pandas as pd
import re
from tqdm import tqdm  
from IPython.display import display
from dateutil.parser import parse

# display all columns in the dataframe
pd.set_option('display.max_columns', None)

# set format for displaying float values
pd.options.display.float_format = '{:20,.7f}'.format

In [2]:
# load loan data
data = pd.read_csv('data/accepted_2007_to_2018Q4.csv.gz', compression='gzip', 
                      header=0, low_memory=False)

# drop the last 2 rows because they contain the total loan amount for policy 1 or 2
data.drop(data.tail(2).index, inplace=True)

## Select Attributes

In [3]:
# select attributes to be included in the study
selected_cols = ['loan_amnt', 'funded_amnt', 'int_rate', 'total_pymnt', 'term', 'issue_d',
                 'last_pymnt_d', 'loan_status', 'desc', 'purpose', 'annual_inc', 'dti', 
                 'revol_bal', 'grade', 'home_ownership', 'delinq_2yrs']
df = data[selected_cols]
df.shape

(2260699, 16)

## Remove Irrelevant Examples

 - Remove In-progress Loans

In [4]:
# get number of loans that are not yet final
df[df['loan_status'].isin(['Current', 'In Grace Period', 'Late (31-120 days)', 'Late (16-30 days)'])].shape

(912569, 16)

In [5]:
# remove loans that are not yet final
df = df[df['loan_status'].isin(['Current', 'In Grace Period', 'Late (31-120 days)', 'Late (16-30 days)']) == False]
df.shape

(1348130, 16)

## Missing Values

 - Get number of missing values for attributes selected in the study.

In [6]:
attributes = []
counts = []
pcts = []

for col in selected_cols:
    c = df[df[col].isnull()].shape[0]    # count number of missing values
    attributes.append(col)
    counts.append(c)
    pcts.append(np.round(c*100/df.shape[0], 4))

missing_vals = pd.DataFrame({'Attribute': attributes, 'Count': counts, 'Percentage': pcts})
missing_vals

,Attribute,Count,Percentage
0,loan_amnt,31,0.0023000
1,funded_amnt,31,0.0023000
2,int_rate,31,0.0023000
3,total_pymnt,31,0.0023000
4,term,31,0.0023000
5,issue_d,31,0.0023000
6,last_pymnt_d,2356,0.1748000
7,loan_status,31,0.0023000
8,desc,1222175,90.6571000
9,purpose,31,0.0023000


 - Exclude attribute "desc" from the study because has ~91% of missing values

In [7]:
selected_cols.remove('desc')

# exclude attribute 'desc' from the study
df = df[selected_cols]
df.shape

(1348130, 15)

 - Remove examples that have missing values since there are very low percentage of missing values

In [8]:
for col in selected_cols:
    df = df[df[col].isna() == False]

# shape of data frame after removing examples that have missing values
df.shape

(1345375, 15)

In [9]:
# check if there are missing values in the data frame
df[df.isnull().any(axis=1)]

,loan_amnt,funded_amnt,int_rate,total_pymnt,term,issue_d,last_pymnt_d,loan_status,purpose,annual_inc,dti,revol_bal,grade,home_ownership,delinq_2yrs


## Change Data Type from Object to Date

In [10]:
# convert data type for date attributes from object to date
for col in ['issue_d', 'last_pymnt_d']:
    df[col] = pd.to_datetime(df[col], format='%b-%Y')

## Derived Attributes

In [11]:
# create a derived attribute, ROI = total payment - funded amount
df['roi'] = df['total_pymnt'] - df['funded_amnt']
selected_cols.append('roi')

# create a derived attribue, months_in_loan = last payment date - loan issued date
df['months_in_loan'] = np.round((df['last_pymnt_d'] - df['issue_d']) / np.timedelta64(1, 'M'), 0)
selected_cols.append('months_in_loan')

# change data type from float to int
df['months_in_loan'] = df['months_in_loan'].astype(int)

In [12]:
# remove total_pymnt and last_pymnt_d
for col in ['total_pymnt', 'last_pymnt_d']:
    selected_cols.remove(col)

df = df[selected_cols]
df.shape

(1345375, 15)

## Binary Attributes

In [13]:
# convert home_ownership to symmetric binary variable: 1 = OWN, 0 = MORTGAGE, RENT, ANY, NONE, OTHER
df['home_ownership'] = df['home_ownership'].apply(lambda x: 1 if 'OWN' in x else 0)

# convert term to symmetric binary variable: 0 = 36 months, 1 = 60 months
df['term'] = df['term'].apply(lambda x: 1 if '60' in x else 0)

## Ranking Ordinal Attribute

In [14]:
# create grade dictionary for mapping
grades = {'A': 7, 'B': 6, 'C': 5, 'D': 4, 'E': 3, 'F': 2, 'G': 1}

df['grade_rank'] = df['grade'].map(grades)
selected_cols.append('grade_rank')

In [15]:
# create loan_status dictionary for mapping
loan_status = {'Fully Paid': 5, 'Does not meet the credit policy. Status:Fully Paid': 4,
               'Charged Off': 3, 'Does not meet the credit policy. Status:Charged Off': 2,
               'Default': 1}
df['loan_status_rank'] = df['loan_status'].map(loan_status)

In [16]:
df.shape

(1345375, 17)

## Create Categorical Attributes

In [17]:
def classifyLoan(roi):
    '''Categorize ROI.'''
    
    roi_range = [0, 2000, 4000]
    labels = ['Unprofitable', 'Low', 'Medium', 'High']
    
    for i, value in enumerate(roi_range):
        if i == 0 and roi <= value:
            # Unprofitable: ROI <= 0
            label = labels[i]
            break
        elif i == len(roi_range) - 1 and roi > value:
            # High: ROI > 4,000
            label = labels[len(labels)-1]
            break
        elif roi > value and roi <= roi_range[i+1]:
            # Low: 0 < ROI <= 2,000
            # Medium: 2,000 < ROI <= 4,000
            label = labels[i+1]
            break
    
    return label

In [18]:
# sort the data by ROI
sorted_data = df.sort_values(by='roi').reset_index()
sorted_data.drop(['index'], axis=1, inplace=True)
sorted_data['loan_return'] = sorted_data['roi'].apply(classifyLoan)

In [19]:
sorted_data.head()

,loan_amnt,funded_amnt,int_rate,term,issue_d,loan_status,purpose,annual_inc,dti,revol_bal,grade,home_ownership,delinq_2yrs,roi,months_in_loan,grade_rank,loan_status_rank,loan_return
0,"40,000.0000000","40,000.0000000",9.9200000,0,2018-03-01,Charged Off,debt_consolidation,"140,000.0000000",10.6700000,"16,312.0000000",B,0,0.0000000,"-39,970.0000000",4,6,3,Unprofitable
1,"40,000.0000000","40,000.0000000",6.1100000,0,2018-10-01,Charged Off,credit_card,"160,000.0000000",11.9000000,"21,623.0000000",A,0,0.0000000,"-39,800.0000000",0,7,3,Unprofitable
2,"40,000.0000000","40,000.0000000",16.9100000,1,2018-07-01,Charged Off,debt_consolidation,"111,000.0000000",18.8400000,"25,834.0000000",C,0,0.0000000,"-39,308.4500000",0,5,3,Unprofitable
3,"40,000.0000000","40,000.0000000",7.3500000,1,2017-12-01,Charged Off,home_improvement,"140,000.0000000",5.9500000,"16,398.0000000",A,1,0.0000000,"-39,217.6600000",1,7,3,Unprofitable
4,"40,000.0000000","40,000.0000000",9.4400000,1,2017-12-01,Charged Off,other,"130,922.0000000",12.5700000,209.0000000,B,0,0.0000000,"-39,182.0700000",1,6,3,Unprofitable


In [20]:
# check if loan_return has missing values
sorted_data['loan_return'].isnull().values.any()

False

In [21]:
# attributes to be categorized
cat_attrs = ['loan_amnt', 'int_rate', 'annual_inc', 'dti', 'revol_bal', 'delinq_2yrs']
categories = [{'low': 10000, 'high': 30000, 'step': 10000},
              {'low': 10, 'high': 20, 'step': 5},
              {'low': 20000, 'high': 100000, 'step': 20000},
              {'low': 5, 'high': 30, 'step': 5},
              {'low': 5000, 'high': 20000, 'step': 5000},
              {'low': 1, 'high': 2, 'step': 1}]
cat_index = 0

In [22]:
def classify_values(valueIn):
    '''
    Categorize continuous variables using low, high, and step value.
    Low value would be categorize as "<= low".
    High value would be categorize as "> high".
    Increased step value would be categorized as "(value1 to value2]"
    
    Parameter:
        valueIn: a numeric value to be categorized
    
    Return a category label.
    '''
    
    # use global variables
    global categories
    global cat_index
    
    # get category spec
    cats_dict = categories[cat_index]
    
    # generate values list
    val_list = list(np.arange(cats_dict['low'], 
                            cats_dict['high'] + cats_dict['step'], 
                            cats_dict['step']))
    
    out = ''
    for i, value in enumerate(val_list):
        value_str = "{:,}".format(value)
        if i == 0 and valueIn <= value:
            out = '<= ' + value_str
            break
        elif i == len(val_list) - 1 and valueIn > value:
            out = '> ' + value_str
            break
        elif valueIn > value and valueIn <= val_list[i+1]:
            out = '(' + value_str + ' to ' + "{:,}".format(val_list[i+1]) + ']'
    
    return out

In [23]:
# categorize continuous variables
for i, attr in enumerate(cat_attrs):
    cat_index = i  # update cat_index (to be used in function classify_values)
    sorted_data[attr + '_cat'] = sorted_data[attr].apply(classify_values)

In [24]:
sorted_data.head()

,loan_amnt,funded_amnt,int_rate,term,issue_d,loan_status,purpose,annual_inc,dti,revol_bal,grade,home_ownership,delinq_2yrs,roi,months_in_loan,grade_rank,loan_status_rank,loan_return,loan_amnt_cat,int_rate_cat,annual_inc_cat,dti_cat,revol_bal_cat,delinq_2yrs_cat
0,"40,000.0000000","40,000.0000000",9.9200000,0,2018-03-01,Charged Off,debt_consolidation,"140,000.0000000",10.6700000,"16,312.0000000",B,0,0.0000000,"-39,970.0000000",4,6,3,Unprofitable,"> 30,000",<= 10,"> 100,000",(10 to 15],"(15,000 to 20,000]",<= 1
1,"40,000.0000000","40,000.0000000",6.1100000,0,2018-10-01,Charged Off,credit_card,"160,000.0000000",11.9000000,"21,623.0000000",A,0,0.0000000,"-39,800.0000000",0,7,3,Unprofitable,"> 30,000",<= 10,"> 100,000",(10 to 15],"> 20,000",<= 1
2,"40,000.0000000","40,000.0000000",16.9100000,1,2018-07-01,Charged Off,debt_consolidation,"111,000.0000000",18.8400000,"25,834.0000000",C,0,0.0000000,"-39,308.4500000",0,5,3,Unprofitable,"> 30,000",(15 to 20],"> 100,000",(15 to 20],"> 20,000",<= 1
3,"40,000.0000000","40,000.0000000",7.3500000,1,2017-12-01,Charged Off,home_improvement,"140,000.0000000",5.9500000,"16,398.0000000",A,1,0.0000000,"-39,217.6600000",1,7,3,Unprofitable,"> 30,000",<= 10,"> 100,000",(5 to 10],"(15,000 to 20,000]",<= 1
4,"40,000.0000000","40,000.0000000",9.4400000,1,2017-12-01,Charged Off,other,"130,922.0000000",12.5700000,209.0000000,B,0,0.0000000,"-39,182.0700000",1,6,3,Unprofitable,"> 30,000",<= 10,"> 100,000",(10 to 15],"<= 5,000",<= 1


## Save Dataframe

In [25]:
sorted_data.to_pickle('data/processed_data.pkl')